#### **Simple Question Answering Using a Transformer Decoder**

We will explore how to train a character-level tranformer language model for a simple question answering task. Given a question of the form `Where was [X] born?` where `[X]` is the name of a public figure, the model will be trained to predict the output `[Y]` which is the name of the birthplace of that person. Our goal is to first pretrain the model on a wikipedia corpus (next character prediction) from which it is expected to acquire knowledge of persons and their birthplaces. Then we finetune the model with supervised training on `(x,y)` sequence pairs of the following form:

`x: Where was Albert Einstein born?%Germany%□□□□□□□□□□□□□□`

`y: □□□□□□□□□□□□□□□□□□□□□□□□□□□□□□%Germany%□□□□□□□□□□□□□□□`

where `x` is the input sequence and `y` is the predicted output sequence and `□` is a special padding token. This is a simple next character prediction task, however we do not want the model to predict the question itself, only the answer, which is why in the output sequence, we replace all characters from the question with the padding token and only have the model predict the characters from the answer. 

i.e. instead of

`y: here was Albert Einstein born?%Germany%□□□□□□□□□□□□□□□`

we use 

`y: □□□□□□□□□□□□□□□□□□□□□□□□□□□□□□%Germany%□□□□□□□□□□□□□□□`

We've also used a special token `%` to mark the beginning and end of the span containing the answer.

**The idea is that by training the model on this task, it can learn to answer a question by retreiving information pertaining to the answer from it's pretrained knowledge.** After training, we can test this idea by giving the model an input sequence which does not contain an answer, i.e. after the start of answer token `%`, we fill the rest of the sequence with padding tokens: 

`x: Where was Enrico Fermi born?%□□□□□□□□□□□□□□□□□□□□□□□□□`

Then if the predicted output sequence contains the right answer, then it will support our idea. We also make sure that person names which were not in the training set will be used during testing.


We will first train the model on the finetuning task without pretraining it and then look at the difference in performance with and without pretraining. 

In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import math
from tqdm import tqdm
import psutil
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tanzids. Use `wandb login --relogin` to force relogin


True

In [39]:
mask_token = u"\u2012"
pad_token = u"\u25A1"

print(f"mask token: {mask_token}")
print(f" token: {pad_token}")

mask token: ‒
 token: □


In [41]:
# first get the character vocabulry from the pretraining dataset
with open("birth_place_data/wiki.txt", 'r') as file:
    pretrain_text = file.read()

vocab = sorted(list(set(pretrain_text)))
assert mask_token not in vocab, "mask token should not be in the vocabulary"
assert pad_token not in vocab, "pad token should not be in the vocabulary"
vocab = vocab + [mask_token, pad_token]
print(f"vocabulary: {vocab}")

vocabulary: ['\n', ' ', '!', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '£', '\xad', 'Á', 'Å', 'É', 'Ó', 'Ö', 'Ø', 'Ü', 'ß', 'à', 'á', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ë', 'í', 'ï', 'ñ', 'ó', 'ô', 'ö', 'ø', 'ü', 'ý', 'ă', 'ą', 'ć', 'Č', 'č', 'ě', 'ğ', 'ī', 'İ', 'ı', 'ł', 'ń', 'ō', 'Ő', 'ő', 'œ', 'ř', 'ś', 'ş', 'Š', 'š', 'ť', 'ū', 'Ż', 'ż', 'Ž', 'ž', 'ș', 'Γ', 'Μ', 'ά', 'έ', 'α', 'γ', 'η', 'ι', 'κ', 'ν', 'ο', 'ρ', 'ς', 'τ', 'υ', 'ω', 'ώ', 'Ј', 'А', 'В', 'Г', 'И', 'К', 'П', 'Р', 'С', 'а', 'б', 'в', 'г', 'д', 'е', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'р', 'с', 'т', 'ц', 'ч', 'ь', 'я', 'ћ', 'א', 'ג', 'ה', 'ו', 'ז', 'ח', 'י', 'כ', 'ל', 'ם', 'מ', 

Create pytorch dataset for finetuning

In [42]:
class NameBirthplaceDataset(Dataset):
    def __init__(self, vocab, mask_token, pad_token, block_size=128, split="train"):
        self.vocab= vocab
        self.ctoi = {c:i for i,c in enumerate(vocab)}
        self.mask_token = mask_token 
        self.pad_token = pad_token
        self.block_size = block_size
        if split == "train":
            data_filename="birth_place_data/birth_places_train.tsv"
        elif split == "dev":
            data_filename="birth_place_data/birth_places_dev.tsv"
    
        self.data = self.read_data(data_filename)
         
    def read_data(self, filename):
        with open(filename, 'r') as f: 
            lines = f.readlines()
        return lines    

    @property
    def pad_token_index(self):
        return self.ctoi[self.pad_token]

    @property
    def mask_token_index(self):
        return self.ctoi[self.mask_token]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        line = self.data[index]
        question, answer = line.strip().split('\t') 
        question, answer = list(question), list(answer)
        x = question + [self.mask_token] + answer + [self.mask_token] 
        x = x + (self.block_size-len(x)) * [self.pad_token] 
        y = [self.pad_token] * (len(question) -1) + x[len(question):]
        x = x[:-1] 

        x = torch.tensor([self.ctoi[c] for c in x], dtype=torch.long)
        y = torch.tensor([self.ctoi[c] for c in y], dtype=torch.long)
        return x, y


In [43]:
train_data = NameBirthplaceDataset(vocab, mask_token, pad_token)
dev_data = NameBirthplaceDataset(vocab, mask_token, pad_token, split="dev")

pad_token_index = train_data.pad_token_index
mask_token_index = train_data.mask_token_index

In [48]:
x, y = train_data[0]

def decode_token_indices(x):
    return "".join([vocab[i] for i in x])

x_decoded = decode_token_indices(x)
y_decoded = decode_token_indices(y)
print(x_decoded)
print(y_decoded)

Where was Khatchig Mouradian born?‒Lebanon‒□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□‒Lebanon‒□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□


#### Create the transformer question answering model

In [73]:
mask = 1-torch.tril(torch.ones(size=(5,5)))
print(mask)
# create mask in which the positions where there are 1s are filled with -infinity 
mask = mask.masked_fill((mask==1), float("-inf"))
print(mask)

tensor([[0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.]])
tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])


In [69]:
class TransformerQA(torch.nn.Module):
    def __init__(self, vocab_size, blocks_size, pad_token_index, embedding_dim=16, feedforward_dim=64, num_heads=1, num_layers=1, dropout_rate=0.1, device="cpu"):
        super().__init__()
        self.vocab_size = vocab_size
        self.block_size = blocks_size
        self.pad_token_index = pad_token_index
        # embedding layers
        self.emb = torch.nn.Embedding(vocab_size, embedding_dim)
        c = 0.01        
        torch.nn.init.uniform_(self.emb.weight, -c, c)

        # static positional encoding (max length set to 1024)
        self.pos_emb = torch.zeros(size=(1024, embedding_dim), device=device)
        for pos in range(1024):
            for i in range(0, embedding_dim, 2):
                self.pos_emb[pos, i] = math.sin(pos / (10000 ** ((2 * i)/embedding_dim)))
                if i+1 < embedding_dim:
                    self.pos_emb[pos, i+1] = math.cos(pos / (10000 ** ((2 * (i+1))/embedding_dim)))

        # transformer decoder
        decoder_layer = torch.nn.TransformerDecoderLayer(d_model=embedding_dim, nhead=num_heads, dim_feedforward=feedforward_dim, activation='gelu', dropout=dropout_rate, batch_first=True)
        self.transformer_decoder = torch.nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        # create output layer (computes output class logits for each item in sequence)
        self.output_layer =  torch.nn.Linear(embedding_dim, vocab_size)
        
        # tie the output layer weights with the embedding layer weights
        #self.output_layer.weight = self.emb.weight


    def create_causal_mask(self, input):
        _, L, _ = input.shape
        # create an L x L matrix with ones on and above diagonal and zero below
        mask = 1-torch.tril(torch.ones(size=(L,L), device=input.device))
        # create mask in which the positions where there are 1s are filled with -infinity 
        mask = mask.masked_fill((mask==1), float("-inf"))
        return mask

    # forward pass
    def forward(self, x, y=None):
        # get embeddings for batch of input sequences of length L
        x = self.emb(x) # shape: (B,L,D)
        # add positional embedding
        x = x + self.pos_emb[:x.shape[1]] # shape: (B,L,D)
        # pass through transformer decoder layers
        mask = self.create_causal_mask(x)
        x = self.transformer_decoder(x, x, tgt_mask=mask) # shape: (B,L,D)
        # compute output logits
        x = self.output_layer(x) # shape: (B,L,vocab_size)

        if y==None:
            return x

        # reshape
        x = x.view(-1,x.shape[-1]) # shape: (B*L,vocab_size)
        y = y.view(-1) # shape: (B*L,)
        # compute cross entropy loss
        loss = F.cross_entropy(x, y, ignore_index=self.pad_token_index)
        return x, loss
    
    """    
    @torch.no_grad()
    def generate(self, subword2idx, block_size, temperature=1.0, topk=None, start_token="<s>", end_token="</s>", max_len=30, device="cpu"):
        self.eval()
        # generate one token at a time
        x = torch.full(size=(1,1), fill_value=subword2idx[start_token], dtype=torch.long, device=device)
        tokens = [x.item()]
        for _ in range(max_len):
            # crop the input sequence so that it doesn't exceed block size (only keep the last block_size tokens in the sequence to generate the next token)
            x = x[:,-block_size:]
            logits = self.forward(x) # shape: (1,L,V)
            # rescale the logits with the temperature
            logits = logits / temperature
            if topk is not None:
                topk_logits, idx = torch.sort(logits[0,-1,:], descending=True)
                # sample from the distribution for the last word in the sequence
                p = F.softmax(topk_logits, dim=-1) # shape: (V,)
                next_word_idx = idx[torch.multinomial(p, num_samples=1)]
            else:             
                # sample from the distribution for the last word in the sequence
                p = F.softmax(logits[:,-1,:], dim=-1) # shape: (V,)
                next_word_idx = torch.multinomial(p, num_samples=1)
            # append to the sequence
            x = torch.cat((x, next_word_idx.view(1,1)), dim=1)
            tokens.append(next_word_idx.item())

        self.train()
        return tokens
        """

# training loop
def train(model, optimizer, scheduler, train_dataloader, val_dataloader, device="cpu", num_epochs=10, val_every=1, save_every=None, log_metrics=None):
    avg_loss = 0
    train_acc = 0
    val_loss = 0
    val_acc = 0
    pp = 0
    for epoch in range(num_epochs):
        num_correct = 0
        num_total = 0
        pbar = tqdm(train_dataloader, desc="Epochs")
        for batch in pbar:
            inputs, targets = batch
            # move batch to device
            inputs, targets = inputs.to(device), targets.to(device)
            # reset gradients
            optimizer.zero_grad()
            # forward pass
            logits, loss = model(inputs, targets)
            # backward pass
            loss.backward()
            # optimizer step
            optimizer.step()
            avg_loss = 0.9* avg_loss + 0.1*loss.item()
            B, L = inputs.shape
            logits = logits.view(B,L,-1)
            y_pred = logits.argmax(dim=-1) # shape (B,L)
            mask = (targets != pad_token_index)
            num_correct += sum([int(torch.allclose(targets[i][mask[i]], y_pred[i][mask[i]])) for i in range(B)])            
            num_total += B
            train_acc = num_correct / num_total        
            
            pbar.set_description(f"Epoch {epoch + 1}, EMA Train Loss: {avg_loss:.3f}, Train Accuracy: {train_acc: .3f}, Val Loss: {val_loss: .3f}, Val Accuracy: {val_acc: .3f}")  

            if log_metrics:
                metrics = {"Batch loss" : loss.item(), "Moving Avg Loss" : avg_loss, "Val Loss": val_loss}
                log_metrics(metrics)

        scheduler.step()
        if epoch%val_every == 0:
            # compute validation loss
            val_loss, val_acc = validation(model, val_dataloader, device=device)
            pbar.set_description(f"Epoch {epoch + 1}, EMA Train Loss: {avg_loss:.3f}, Train Accuracy: {train_acc: .3f}, Val Loss: {val_loss: .3f}, Val Accuracy: {val_acc: .3f}") 

        if save_every is not None:
            if (epoch+1) % save_every == 0:
                save_model_checkpoint(model, optimizer, epoch, avg_loss)

def validation(model, val_dataloader, device="cpu"):
    model.eval()
    val_losses = torch.zeros(len(val_dataloader))
    with torch.no_grad():
        num_correct = 0
        num_total = 0
        for i,batch in enumerate(val_dataloader):
            inputs, targets = batch = batch
            inputs, targets = inputs.to(device), targets.to(device)
            B, L = inputs.shape
            logits, loss = model(inputs, targets)
            logits = logits.view(B,L,-1)
            y_pred = logits.argmax(dim=-1) # shape (B,L)
            mask = (targets != pad_token_index)
            num_correct += sum([int(torch.allclose(targets[i][mask[i]], y_pred[i][mask[i]])) for i in range(B)])            
            num_total += B
            val_losses[i] = loss.item()
    model.train()
    val_loss = val_losses.mean().item()
    val_accuracy = num_correct / num_total
    return val_loss, val_accuracy


def evaluate(model, dataloader, device="cpu"):
        model.eval()
        with torch.no_grad():
            batch = next(iter(dataloader))
            inputs, targets = batch = batch
            inputs, targets = inputs.to(device), targets.to(device)
            B, L = inputs.shape
            logits, loss = model(inputs, targets)
            y_pred = logits.argmax(dim=-1) # shape (B*L)
            y_pred = y_pred.view(B,L)
        model.train()
        return inputs, targets, y_pred

# sample a sequence from the model
def sample(model, x, block_size=128, sample=False, device="cpu"):
    model.eval()
    with torch.no_grad():
        question_length = len(x.view(-1))
        x = x.to(device)
        for _ in range(block_size-question_length):
            # crop the input sequence so that it doesn't exceed block size (only keep the last block_size tokens in the sequence to generate the next token)
            x = x[:,-block_size:]
            logits = model(x) # shape: (1,L,V)
            # sample from the distribution to get the next character
            p = F.softmax(logits[:,-1,:], dim=-1) # shape: (V,)
            if sample:
                next_char_idx = torch.multinomial(p, num_samples=1)
            else:
                _, next_char_idx = torch.topk(p, k=1)
            # append to the sequence
            x = torch.cat((x, next_char_idx.view(1,1)), dim=1)
    model.train()
    return x


def save_model_checkpoint(model, optimizer, epoch=None, loss=None):
    # Save the model and optimizer state_dict
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }

    # Save the checkpoint to a file
    torch.save(checkpoint, 'qa_model_checkpoint.pth')
    print(f"Saved model checkpoint!")


def load_model_checkpoint(model, optimizer):
    checkpoint = torch.load('qa_model_checkpoint.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    model.train()
    print("Loaded model from checkpoint!")
    return model, optimizer      

In [70]:
B = 32
D = 256
vocab_size = len(vocab)
num_heads = 8
num_layers = 4
learning_rate = 1e-4
DEVICE = "cuda"

train_dataloader = DataLoader(train_data, batch_size=B, shuffle=True, pin_memory=True, num_workers=2)
val_dataloader = DataLoader(dev_data, batch_size=B, shuffle=True, pin_memory=True, num_workers=2)


model = TransformerQA(vocab_size, blocks_size=128, pad_token_index=train_data.pad_token_index, embedding_dim=D, feedforward_dim=4*D, num_heads=num_heads, num_layers=num_layers, dropout_rate=0.2, device=DEVICE).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.95)
#model, optimizer = load_model_checkpoint(model, optimizer)


num_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters in transformer network: {num_params/1e6} M")
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

Total number of parameters in transformer network: 4.345088 M
RAM used: 1186.20 MB


In [71]:
train(model, optimizer, scheduler, train_dataloader, val_dataloader, device=DEVICE, num_epochs=100, save_every=10, val_every=1) #, log_metrics=log_metrics)

Epoch 1, EMA Train Loss: nan, Train Accuracy:  0.000, Val Loss:  0.000, Val Accuracy:  0.000: 100%|██████████| 63/63 [00:03<00:00, 18.70it/s]
Epoch 2, EMA Train Loss: nan, Train Accuracy:  0.000, Val Loss:  nan, Val Accuracy:  0.000: 100%|██████████| 63/63 [00:02<00:00, 21.74it/s]
Epoch 3, EMA Train Loss: nan, Train Accuracy:  0.000, Val Loss:  nan, Val Accuracy:  0.000:  29%|██▊       | 18/63 [00:00<00:02, 19.92it/s]


KeyboardInterrupt: 

In [65]:
inputs, targets, y_pred = evaluate(model, train_dataloader, device=DEVICE)
for i in range(5):
    x = inputs[i]
    y = targets[i]
    y_hat = y_pred[i]
    print(f"Input:      {decode_token_indices(x)}")
    print(f"Target:     {decode_token_indices(y)}")
    print(f"Prediction: {decode_token_indices(y_hat)}")
    print("")

Input:      Where was Patrick Lemarié born?‒Paris‒□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
Target:     □□□□□□□□□□□□□□□□□□□□□□□□□□□□□□‒Paris‒□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
Prediction: ‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒MoMlkhhornt‒Paris‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒

Input:      Where was Brett Hayman born?‒Melbourne‒□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
Target:     □□□□□□□□□□□□□□□□□□□□□□□□□□□‒Melbourne‒□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
Prediction: ‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒HA‒ppeuWnd‒Melbourne‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒

Input:      Where was Réginald Bernut born?‒Sydney‒□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
Target:     □□□□□□

Generate some sequences continuing from questions from the training set

In [63]:
for i in range(len(inputs)):
    x = inputs[i]
    x = x[:torch.where(x == mask_token_index)[0][0]+1]
    y = targets[i]
    y_pred = sample(model, x.view(1,-1), device=DEVICE)
    print(f"Input:      {decode_token_indices(x)}")
    print(f"Target:     {decode_token_indices(y)}")
    print(f"Prediction: {decode_token_indices(y_pred[0])}")
    print("")

Input:      Where was Alec Briggs born?‒
Target:     □□□□□□□□□□□□□□□□□□□□□□□□□□‒Sheffield‒□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
Prediction: Where was Alec Briggs born?‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒

Input:      Where was Maurice Vaïsse born?‒
Target:     □□□□□□□□□□□□□□□□□□□□□□□□□□□□□‒Algiers‒□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
Prediction: Where was Maurice Vaïsse born?‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒

Input:      Where was Gonzalo Pieres, Sr. born?‒
Target:     □□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□‒Argentina‒□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
Prediction: Where was Gonzalo Pieres, Sr. born?‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒

Input:      Where wa